<a href="https://colab.research.google.com/github/BrianChuan/TAICA_AI-Text-and-Image/blob/main/%E5%8C%97%E7%A7%91%E5%A4%A7_111360205_%E5%AD%90%E5%9B%9B%E4%B9%99_%E8%AC%9D%E9%80%B2%E6%AC%8A_week2_%E6%89%8B%E5%AF%AB%E8%BE%A8%E8%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 建立AI手寫辨識模型
- 本學期第一個神經網路(Neual Network)
- 考慮到資料複雜度不高，避免overfitting，嘗試製作 5 層的深度學習模型。
- 程式碼中，以 變數 `N1` 表示第一層有 N1 個神經元，其餘以此類推。


In [ ]:
N1 = 30
N2 = 25
N3 = 20
N4 = 20
N5 = 20

## 1. 讀入各種套件
### 數據分析、畫圖
- Numpy：主要用於數值運算，特色是可以提供多維陣列物件(ndarray)、向量化的開源python library。
- matplotlib：呈現結果的圖片生成。
- Pillow(PIL)：python的影像處理函式庫，讓python擁有程式碼讀取、操作和儲存各種圖片格式的能力。

### 神經網路
- tensorflow：由Google 開發的學習框架，可以處理所有繁重計算，像是Tensors, gradient在不同硬體上的計算。
    - keras：使用者介面(API)，讓使用者快速直觀的定義結構、編譯和訓練模型。

### 互動設計
- ipywidgets：Jupyter Notebook 的一個擴充工具箱，它將靜態的程式碼轉化為動態、互動式的應用程式，本次實作使用其中資料探索的功能。
- interact_manual：Jupyter Notebbok中的互動控制項，只要改變控制項如滑桿，就能觸發函式執行並依據輸入更新輸出。

#### 網頁
- Gradio：主要功能可以快速為機器學習模型建立一個簡單的UI，製作成一個互動式網頁應用。

In [ ]:
!pip install gradio

In [ ]:
%matplotlib inline

import pandas as pd

# 標準數據分析、畫圖套件
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# 神經網路方面
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

# 互動設計用
from ipywidgets import interact_manual

# 神速打造 web app 的 Gradio
import gradio as gr

## 2. 讀入 MNIST 數據庫
> MNIST: Modified National Institude of Standards and Technology

### MNIST 介紹
- NIST代表美國國家標準與技術研究院，最初的數據集由此單位提供。
- 名稱中的"M"表示"Modified"也就是調整過的。[網頁](http://yann.lecun.com/exdb/mnist/)
- 其中0-9的手寫數字圖庫有 `6 萬`筆訓練資料，`1 萬`筆測試資料。
- 可以說是Deep Learning中最有名的範例資料集。

### 2.1 由 Keras 讀入 MNIST

> 利用Keras可以快速讀入 MNIST 的資料集。

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

利用print指令，來確定訓練資料集為 6萬 筆資料，測試資料集為 1萬 筆資料。

In [ ]:
print(f'訓練資料總筆數為 {len(x_train)} 筆資料')
print(f'測試資料總筆數為 {len(x_test)} 筆資料')

訓練資料總筆數為 60000 筆資料
測試資料總筆數為 10000 筆資料


### 2.2 數據庫的內容
- 輸入(x)：手寫數字0~9的圖片
- 輸出(y)：圖片對應的正確答案

In [ ]:
def show_xy(n=0):
    ax = plt.gca()
    X = x_train[n]
    plt.xticks([], [])
    plt.yticks([], [])
    plt.imshow(X, cmap = 'Greys')
    print(f'本資料 y 給定的答案為: {y_train[n]}')

In [ ]:
interact_manual(show_xy, n=(0,59999));

interactive(children=(IntSlider(value=0, description='n', max=59999), Button(description='Run Interact', style…

In [ ]:
def show_data(n = 100):
    X = x_train[n]
    print(X)

In [ ]:
interact_manual(show_data, n=(0,59999));

interactive(children=(IntSlider(value=100, description='n', max=59999), Button(description='Run Interact', sty…

### 2.3 輸入格式調整
- 目的：將輸入資料可以符合標準神經網路的格式。
- 使用函式：`reshape`
- 調整內容：將原本$28\times 28$的矩陣，轉換為長度為$784$的向量($28\times 28 = 784$)。

In [ ]:
# 一定要執行，用於整理資料
x_train = x_train.reshape(60000, 784)/255
x_test = x_test.reshape(10000, 784)/255

### 2.4 輸出格式整理

- 想學的函數形式為 $\hat{f} \colon \mathbb{R}^{784} \to \mathbb{R}$
    - 一個$784$個任意實數所組成的的向量，對應到一個實數。

- 問題：神經網路輸出的結果可能為小數，此時就無法判別結果。
    - Ex.$\hat{f}(x) = 0.5$

- 解決方式：`1-hot enconding`，讓DNN有確定的輸出。
    - Ex. 數字 1 變為 [0, 1, 0, 0, 0, 0, 0, 0, 0]、 數字 5 變為 [0, 0, 0, 0, 0, 1, 0, 0, 0]
    - 可利用 Keras 套件做到這件事。

In [ ]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

## 3. 打造第一個神經網路

- 目標函數為 $\hat{f} \colon \mathbb{R}^{784} \to \mathbb{R}^{10}$
- 要決定的參數：隱藏層數量、每層的神經元數量、用哪個激發函數。

### 3.1 決定神經網路架構、讀入相關套件
- 激發函數：ReLU
- 設計方式：告訴TensorFlow

### 3.2 建構神經網路
- `Sequential`：標準一層一層的神經網路。

In [ ]:
model = Sequential()

第一層的隱藏層因為要符合輸入資料集，所以要設定`input_dim`

In [ ]:
model.add(Dense(N1, input_dim=784, activation='relu'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


第二層開始就不用再說明輸入神經元個數 (因為就是前一層神經元數)。

In [ ]:
model.add(Dense(N2, activation='relu'))
model.add(Dense(N3, activation='relu'))
model.add(Dense(N4, activation='relu'))
model.add(Dense(N5, activation='relu'))

輸出可能有10種結果數字(0~9)，所以對應到輸出層神經元有10個

輸出條件：
1. $(y_1, y_2, \ldots, y_{10})$：每個$y_i$表示手寫數字為$i$的機率。
2. $\sum_{i=1}^{10} y_i = 1$：要滿足的機率特性，所有發生的可能總合要為1。

 使用 `softmax` 當激發函數就能滿足以上條件

In [ ]:
# 輸出層，數量要是輸出的數量
model.add(Dense(10, activation='softmax'))

### 3.3 組裝

- 步驟重點：完成compile，才算正式將神經網路建好。

#### 缺少的項目:
* loss function: 選用`mse`，雖然outler容易造成影響。
* optimizer: SGD (Stochastic Gradient Descent) 稱隨機梯度下降。
* 設定 learning rate: 也就是調整參數的幅度。
> 若是想讓訓練同時看到準確率，加入指令，就能同步出現。
> metric = ['accuracy']

In [ ]:
# mse: 平方差。SGD: 隨機型的gradient
model.compile(loss='mse', optimizer=SGD(learning_rate=0.087), metrics=['accuracy'])

## 4. 檢視並確認神經網路

> 此步驟主要用於，確定所設定的模型參數，和想象中相同

### 4.1 使用 model 的 summary 查看以上設定的所有內容
> 主要檢查項目為參數數量

In [ ]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 30)             │        23,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 25)             │           775 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 20)             │           520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 10)             │           210 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,895 (101.15 KB)

 Trainable params: 25,895 (101.15 KB)

 Non-trainable params: 0 (0.00 B)

## 5. 開始訓練神經網路

訓練時，人工需要提供的參數:
* 一次要訓練幾筆資料 (`batch_size`)
    * 設 batch_size=100 ，代表每訓練 100 筆資料，調一次參數。
* 這 6 萬筆資料一共要訓練幾次稱為 (`epochs`)
    * 設為 10 代表 6萬筆資料總共訓練10次


In [ ]:
# model.fit(x_train, y_train, batch_size=100, epochs=20) --> 正確率 91%
# model.fit(x_train, y_train, batch_size=100, epochs=30) --> 正確率 93.5%
model.fit(x_train, y_train, batch_size=90, epochs=50) # 正確率 97% -> overfit

Epoch 1/50
667/667 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9646 - loss: 0.0058
Epoch 2/50
667/667 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9674 - loss: 0.0055
Epoch 3/50
667/667 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9670 - loss: 0.0055
Epoch 4/50
667/667 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9670 - loss: 0.0056
Epoch 5/50
667/667 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9667 - loss: 0.0056
Epoch 6/50
667/667 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9683 - loss: 0.0054
Epoch 7/50
667/667 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9684 - loss: 0.0053
Epoch 8/50
667/667 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9694 - loss: 0.0052
Epoch 9/50
667/667 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9695 - loss: 0.0052
Epoch 10/50
667/667 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9683 - loss: 0.0053
Epoch 11/50
667/667 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9693 - loss: 0.0052
Epoch 12/50
667/667 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step

## 6. 結果


In [ ]:
loss, acc = model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9493 - loss: 0.0081


In [ ]:
print(f"測試資料正確率 {acc*100:.2f}%")

測試資料正確率 95.49%


- "predict" 放神經網路訓練後的結果，並且使用 argmax 找到信心度最高的項目。

In [ ]:
predict = np.argmax(model.predict(x_test), axis=-1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


- 由於先前的 reshape，讓資料變成 784 維的向量，因此要轉換為 28x28 的矩陣才能正常顯示出圖片。

In [ ]:
def test(測試編號):
    plt.imshow(x_test[測試編號].reshape(28,28), cmap='Greys')
    print('神經網路判斷為:', predict[測試編號])

In [ ]:
interact_manual(test, 測試編號=(0, 9999));

interactive(children=(IntSlider(value=4999, description='測試編號', max=9999), Button(description='Run Interact', …

### 總評分評量：

In [ ]:
score = model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9493 - loss: 0.0081


In [ ]:
print('loss:', score[0])
print('正確率', score[1])

loss: 0.007251115515828133
正確率 0.9549000263214111


### 7. Gradio 展示

In [ ]:
def resize_image(inp):
    # 圖在 inp["layers"][0]
    image = np.array(inp["layers"][0], dtype=np.float32)
    image = image.astype(np.uint8)

    # 轉成 PIL 格式
    image_pil = Image.fromarray(image)

    # Alpha 通道設為白色, 再把圖從 RGBA 轉成 RGB
    background = Image.new("RGB", image_pil.size, (255, 255, 255))
    background.paste(image_pil, mask=image_pil.split()[3]) # 把圖片粘貼到白色背景上，使用透明通道作為遮罩
    image_pil = background

    # 轉換為灰階圖像
    image_gray = image_pil.convert("L")

    # 將灰階圖像縮放到 28x28, 轉回 numpy array
    img_array = np.array(image_gray.resize((28, 28), resample=Image.LANCZOS))

    # 配合 MNIST 數據集
    img_array = 255 - img_array

    # 拉平並縮放
    img_array = img_array.reshape(1, 784) / 255.0

    return img_array

In [ ]:
def recognize_digit(inp):
    img_array = resize_image(inp)
    prediction = model.predict(img_array).flatten()
    labels = list('0123456789')
    return {labels[i]: float(prediction[i]) for i in range(10)}

In [ ]:
iface = gr.Interface(
    fn=recognize_digit,
    inputs=gr.Sketchpad(),
    outputs=gr.Label(num_top_classes=3),
    title="MNIST 手寫辨識",
    description="請在畫板上繪製數字"
)

iface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://348556112868db65b3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://348556112868db65b3.gradio.live
